# Tabular Data RAG

In [30]:
import os
from dotenv import load_dotenv
import pandas as pd

# Required to convert to markdown
#%pip install tabulate

# OpenAI dependencies
#%pip install openai==1.55.3
from openai import OpenAI

#%pip install azure-identity
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

## Load Datasets

In [31]:
# years of evaluation
start_year = 2019
end_year = 2023

# datasets files
teams_cleaned_dataset_file = os.path.join("datasets", "teams_clean.csv")
standings_cleaned_dataset_file = os.path.join("datasets" ,"standings_clean.csv")
games_cleaned_dataset_file = os.path.join("datasets", "games_clean.csv")

# seasons to evaluate
seasons_to_evaluate = [year for year in range(start_year, end_year + 1)]

# read standings dataset
standings = pd.read_csv(standings_cleaned_dataset_file)

standings = standings.drop(standings[~standings["season"].isin(seasons_to_evaluate)].index)

In [32]:
# filder standings by team name NE 
#standings_filtered = standings[standings['team'] == 'NE']

# convert standings to markdown
standings_markdown = standings.to_markdown()

print(standings_markdown)

|     |   season | conf   | division   | team   |   wins |   losses |   ties |      pct |   div_rank |   scored |   allowed |   net |   seed | playoff   | team_name             |
|----:|---------:|:-------|:-----------|:-------|-------:|---------:|-------:|---------:|-----------:|---------:|----------:|------:|-------:|:----------|:----------------------|
| 160 |     2019 | AFC    | AFC East   | BUF    |     10 |        6 |      0 | 0.625    |          2 |      314 |       259 |    55 |      5 | LostWC    | Buffalo Bills         |
| 161 |     2019 | AFC    | AFC East   | MIA    |      5 |       11 |      0 | 0.3125   |          4 |      306 |       494 |  -188 |    nan | nan       | Miami Dolphins        |
| 162 |     2019 | AFC    | AFC East   | NE     |     12 |        4 |      0 | 0.75     |          1 |      420 |       225 |   195 |      3 | LostWC    | New England Patriots  |
| 163 |     2019 | AFC    | AFC East   | NYJ    |      7 |        9 |      0 | 0.4375   |          3 |   

## Run Language Model

In [33]:
MESSAGE_SYSTEM_CONTENT = """You are a customer service agent that helps a customer with answering questions about NFL statistical data. 
Please answer the question based on the provided context below.
Te context is a table in markdown format that contains NFL standings data for the 2019-2023 seasons.
Make sure not to make any changes to the context, if possible, when preparing answers to provide accurate responses. 
If the answer cannot be found in context, just politely say that you do not know, do not try to make up an answer."""

In [34]:
load_dotenv(override=True)
# Get environment variables
project_endpoint = os.getenv("PROJECT_ENDPOINT")
model_deployment = os.getenv("MODEL_DEPLOYMENT")
api_key = os.getenv("API_KEY")

token_provider = get_bearer_token_provider(  
    DefaultAzureCredential(),  
    "https://cognitiveservices.azure.com/.default"  
)
client = OpenAI(
    api_key=token_provider,
    base_url=project_endpoint)

In [35]:

def response_test(question:str, context:str, model:str = "gpt-4o"):
    response = client.responses.create(
        model=model,
        instructions=MESSAGE_SYSTEM_CONTENT,
        input=[            
            {"role": "user", "content": question},
            {"role": "user", "content": context},
        ],
    )
    
    return response.model_dump_json(indent=2)

In [36]:
response = response_test("What is the div_rank for the 2020 NFL season for New England Patriots?", standings_markdown, model_deployment)
print(response)

{
  "id": "resp_07b59bbb71cba37200695d32d4f0888190b4eda06ff886d15e",
  "created_at": 1767715540.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "You are a customer service agent that helps a customer with answering questions about NFL statistical data. \nPlease answer the question based on the provided context below.\nTe context is a table in markdown format that contains NFL standings data for the 2019-2023 seasons.\nMake sure not to make any changes to the context, if possible, when preparing answers to provide accurate responses. \nIf the answer cannot be found in context, just politely say that you do not know, do not try to make up an answer.",
  "metadata": {},
  "model": "gpt-4o-mini",
  "object": "response",
  "output": [
    {
      "id": "msg_07b59bbb71cba37200695d32d57ed08190a414b7e03adacaca",
      "content": [
        {
          "annotations": [],
          "text": "The div_rank for the New England Patriots in the 2020 NFL season is 3.",
          "typ

In [37]:
response = response_test("What is the average of scored points for New England Patriots?", standings_markdown, model_deployment)
print(response)

{
  "id": "resp_0c80022f4835037b00695d32d62364819083452b41264577b9",
  "created_at": 1767715542.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "You are a customer service agent that helps a customer with answering questions about NFL statistical data. \nPlease answer the question based on the provided context below.\nTe context is a table in markdown format that contains NFL standings data for the 2019-2023 seasons.\nMake sure not to make any changes to the context, if possible, when preparing answers to provide accurate responses. \nIf the answer cannot be found in context, just politely say that you do not know, do not try to make up an answer.",
  "metadata": {},
  "model": "gpt-4o-mini",
  "object": "response",
  "output": [
    {
      "id": "msg_0c80022f4835037b00695d32d6a908819094609aea8fb90270",
      "content": [
        {
          "annotations": [],
          "text": "To calculate the average scored points for the New England Patriots over the seasons pr

In [38]:
response = response_test("Which team has the best scored points average in the seasons provided?", standings_markdown, model_deployment)
print(response)

{
  "id": "resp_066253d3f842c7ef00695d32d8d8948196984409a8a45bb9ae",
  "created_at": 1767715544.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "You are a customer service agent that helps a customer with answering questions about NFL statistical data. \nPlease answer the question based on the provided context below.\nTe context is a table in markdown format that contains NFL standings data for the 2019-2023 seasons.\nMake sure not to make any changes to the context, if possible, when preparing answers to provide accurate responses. \nIf the answer cannot be found in context, just politely say that you do not know, do not try to make up an answer.",
  "metadata": {},
  "model": "gpt-4o-mini",
  "object": "response",
  "output": [
    {
      "id": "msg_066253d3f842c7ef00695d32d93f848196b359e2382281ffea",
      "content": [
        {
          "annotations": [],
          "text": "I don't have that information based on the provided context.",
          "type": "outpu

In [39]:
response = response_test("What is the team with more wins in the data provided?", standings_markdown, model_deployment)
print(response)

{
  "id": "resp_0684b1d1a4afaae800695d32d9cf0c8197bddf27fe386a7107",
  "created_at": 1767715545.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "You are a customer service agent that helps a customer with answering questions about NFL statistical data. \nPlease answer the question based on the provided context below.\nTe context is a table in markdown format that contains NFL standings data for the 2019-2023 seasons.\nMake sure not to make any changes to the context, if possible, when preparing answers to provide accurate responses. \nIf the answer cannot be found in context, just politely say that you do not know, do not try to make up an answer.",
  "metadata": {},
  "model": "gpt-4o-mini",
  "object": "response",
  "output": [
    {
      "id": "msg_0684b1d1a4afaae800695d32daa8f08197abb8ae42a018c7b5",
      "content": [
        {
          "annotations": [],
          "text": "The team with the most wins in the provided data is the Baltimore Ravens, with 14 wins 

In [40]:
response = response_test("What is the team with less total wins in the provided seasons?", standings_markdown, model_deployment)
print(response)

{
  "id": "resp_0a9301f0b34de52b00695d32dcf250819395a0407cb02373dd",
  "created_at": 1767715548.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "You are a customer service agent that helps a customer with answering questions about NFL statistical data. \nPlease answer the question based on the provided context below.\nTe context is a table in markdown format that contains NFL standings data for the 2019-2023 seasons.\nMake sure not to make any changes to the context, if possible, when preparing answers to provide accurate responses. \nIf the answer cannot be found in context, just politely say that you do not know, do not try to make up an answer.",
  "metadata": {},
  "model": "gpt-4o-mini",
  "object": "response",
  "output": [
    {
      "id": "msg_0a9301f0b34de52b00695d32dd5f9481939e846e904acde6ba",
      "content": [
        {
          "annotations": [],
          "text": "The team with the least total wins across the provided seasons is the **Carolina Panthe